In [1]:
import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

import time, re

In [2]:
urls = ['https://www.bestbuy.com/site/mcafee-total-protection-5-device-antivirus-internet-security-software-1-year-subscription-android-apple-ios-chrome-mac-os-windows-digital/6517326.p?skuId=6517326',
        'https://www.bestbuy.com/site/norton-360-deluxe-5-device-antivirus-internet-security-software-vpn-dark-web-monitoring-1-year-subscription-android-mac-os-windows-apple-ios-digital/6346690.p?skuId=6346690',
        'https://www.bestbuy.com/site/jbl-flip-5-portable-bluetooth-speaker-black/6356535.p?skuId=6356535',
        'https://www.bestbuy.com/site/apple-iphone-13-5g-128gb-unlocked-midnight/6417788.p?skuId=6417788',
        'https://www.bestbuy.com/site/samsung-galaxy-s23-ultra-256gb-unlocked-phantom-black/6529723.p?skuId=6529723']

s = Service(executable_path='D:\CU Boulder - Data Science\Spring 2024\Data Mining\chromedriver.exe')

In [3]:
all_star_rating = []
all_review = []
all_name = []
all_review_title = []
all_votes = []

In [4]:
star_rating = []
review = []
name = []
review_title = []
votes = []

driver = webdriver.Chrome(service=s)
driver.get(urls[2])
driver.maximize_window()
time.sleep(3)

reviews_button = driver.find_element(By.XPATH,'/html/body/div[4]/main/div[5]/div[2]/div[1]/div/div/div/div/div/div[4]/div/div/div/div[6]/a[1]')
reviews_button.click()
time.sleep(3)

review_div = driver.find_elements(By.CLASS_NAME,'review-item')

tot_reviews = driver.find_element(By.XPATH, '/html/body/div[2]/main/div[1]/div/div/div[1]/div/div/div[3]/div[2]/div/div[1]/div[2]/div/div/div/ul/li[1]/a/div/span[2]').text

match = re.search(r'\(([\d,]+)\sReviews\)', tot_reviews)

if match:
    review_count_str = match.group(1)
    # Remove commas from the string and convert to integer
    review_count = int(review_count_str.replace(',', ''))
    print("Number of reviews:", review_count)
else:
    print("Number of reviews not found")

num_pages = round(review_count/20)

for i in range(1,num_pages):
    try:
        # Retrieve review elements
        review_elements = driver.find_elements(By.XPATH, '//*[@id="reviews-accordion"]/div[1]/ul/li')
        total_reviews = len(review_elements)

        # Loop through each review on the current page
        for i in range(1, total_reviews + 1):
            try:
                star = driver.find_element(By.XPATH, f'//*[@id="reviews-accordion"]/div[1]/ul/li[{i}]/div/div[2]/div[1]/div/div/p').text[6:7]
                rev = driver.find_element(By.XPATH, f'/html/body/div[2]/main/div[1]/div/div/div[2]/div/div/div/div/div[1]/ul/li[{i}]/div/div[2]/div[5]/p').text
                rev_name = driver.find_element(By.XPATH, f'//*[@id="reviews-accordion"]/div[1]/ul/li[{i}]/div/div[1]/div/span/span/button/div').text
                title = driver.find_element(By.XPATH, f'/html/body/div[2]/main/div[1]/div/div/div[2]/div/div/div/div/div[1]/ul/li[{i}]/div/div[2]/div[1]/h4').text
                helpful_votes = driver.find_element(By.XPATH, f'/html/body/div[2]/main/div[1]/div/div/div[2]/div/div/div/div/div[1]/ul/li[{i}]/div/div[2]/div[7]/div/div/div[1]/button[1]').text.split('(')[1].split(')')[0]
            except Exception as e:
                print(e)

            star_rating.append(star)
            review.append(rev)
            name.append(rev_name)
            review_title.append(title)
            votes.append(helpful_votes)
        
        # Find and click the button to navigate to the next page
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, '#reviews-accordion > div.page-pagination.mb-300 > div > div:nth-child(2) > ul > li.page.next > a'))
        )
        next_button.click()
        time.sleep(3)

        all_star_rating.extend(star_rating)
        all_review.extend(review)
        all_name.extend(name)
        all_review_title.extend(review_title)
        all_votes.extend(votes)
    
    except NoSuchElementException:
        # If no next button is found, exit the loop
        break

Number of reviews: 5900
Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[2]/main/div[1]/div/div/div[2]/div/div/div/div/div[1]/ul/li[1]/div/div[2]/div[5]/p"}
  (Session info: chrome=122.0.6261.69); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6F6C05E42+3538674]
	(No symbol) [0x00007FF6F6824C02]
	(No symbol) [0x00007FF6F66D5AEB]
	(No symbol) [0x00007FF6F671BF4E]
	(No symbol) [0x00007FF6F671C0CC]
	(No symbol) [0x00007FF6F675E477]
	(No symbol) [0x00007FF6F673F0EF]
	(No symbol) [0x00007FF6F675BDE3]
	(No symbol) [0x00007FF6F673EE53]
	(No symbol) [0x00007FF6F670F514]
	(No symbol) [0x00007FF6F6710631]
	GetHandleVerifier [0x00007FF6F6C36CAD+3738973]
	GetHandleVerifier [0x00007FF6F6C8C506+4089270]
	GetHandleVerifier [0x00007FF6F6C84823+4057299]
	GetHandleVerifier [0x00007FF6F6955C49+720121]
	(No symbol) [0x000

KeyboardInterrupt: 

In [5]:
reviews_df = pd.DataFrame({'Reviewer_Name':name,'Review_Title':review_title, 'Review_content':review ,'Star_Rating':star_rating})
reviews_df.head()

,Reviewer_Name,Review_Title,Review_content,Star_Rating
0,Luisitolammatao112,Wow… superb,Jbl flip 5 is superd and the price. The sounds...,5
1,NeoSheridan,Excellent sound and portablilty,"""Excellent sound, portability and unity"" \nthi...",5
2,GamingKing,Perfect pool and beach speaker,"Speaker is nice and loud, clear, and water pro...",5
3,Anonymous,JBL 5 compact portable speaker,This is my favorite brand on portable speakers...,5
4,Anonymous,JBL flip,Im all into music and this speaker is best For...,5


In [6]:
len(reviews_df)

520

In [7]:
driver.quit()

In [4]:
def get_total_pages(element_total_reviews):

    match = re.search(r'\((\d{2,4})\sReviews\)', element_total_reviews)
    review_count = match.group(1)
    count = int(review_count)
    num_pages = round(count/20)

    return num_pages

In [7]:
driver = webdriver.Chrome(service=s)
driver.get(urls[0])
driver.maximize_window()
time.sleep(3)

reviews_button = driver.find_element(By.XPATH,'/html/body/div[4]/main/div[5]/div[2]/div[1]/div/div/div/div/div/div[4]/div/div/div/div[6]/a[1]')
reviews_button.click()
time.sleep(3)

review_div = driver.find_elements(By.CLASS_NAME,'review-item')

tot_reviews = driver.find_element(By.XPATH, '/html/body/div[2]/main/div[1]/div/div/div[1]/div/div/div[3]/div[2]/div/div[1]/div[2]/div/div/div/ul/li[1]/a/div/span[2]').text
n = get_total_pages(tot_reviews)
n

12

In [5]:
all_star_rating = []
all_review = []
all_name = []
all_review_title = []
all_votes = []

In [6]:
for i in urls[1]:

    star_rating = []
    review = []
    name = []
    review_title = []
    votes = []

    driver = webdriver.Chrome(service=s)
    driver.get(i)
    driver.maximize_window()
    time.sleep(3)

    reviews_button = driver.find_element(By.XPATH,'/html/body/div[4]/main/div[5]/div[2]/div[1]/div/div/div/div/div/div[4]/div/div/div/div[6]/a[1]')
    reviews_button.click()
    time.sleep(3)

    review_div = driver.find_elements(By.CLASS_NAME,'review-item')

    tot_reviews = driver.find_element(By.XPATH, '/html/body/div[2]/main/div[1]/div/div/div[1]/div/div/div[3]/div[2]/div/div[1]/div[2]/div/div/div/ul/li[1]/a/div/span[2]').text
    n = get_total_pages(tot_reviews)

    for i in range(1,n):
        try:
            # Retrieve review elements
            review_elements = driver.find_elements(By.XPATH, '//*[@id="reviews-accordion"]/div[1]/ul/li')
            total_reviews = len(review_elements)

            # Loop through each review on the current page
            for i in range(1, total_reviews + 1):
                try:
                    star = driver.find_element(By.XPATH, f'//*[@id="reviews-accordion"]/div[1]/ul/li[{i}]/div/div[2]/div[1]/div/div/p').text[6:7]
                    rev = driver.find_element(By.XPATH, f'/html/body/div[2]/main/div[1]/div/div/div[2]/div/div/div/div/div[1]/ul/li[{i}]/div/div[2]/div[5]/p').text
                    rev_name = driver.find_element(By.XPATH, f'//*[@id="reviews-accordion"]/div[1]/ul/li[{i}]/div/div[1]/div/span/span/button/div').text
                    title = driver.find_element(By.XPATH, f'/html/body/div[2]/main/div[1]/div/div/div[2]/div/div/div/div/div[1]/ul/li[{i}]/div/div[2]/div[1]/h4').text
                    helpful_votes = driver.find_element(By.XPATH, f'/html/body/div[2]/main/div[1]/div/div/div[2]/div/div/div/div/div[1]/ul/li[{i}]/div/div[2]/div[7]/div/div/div[1]/button[1]').text.split('(')[1].split(')')[0]
                except Exception as e:
                    print(e)

                star_rating.append(star)
                review.append(rev)
                name.append(rev_name)
                review_title.append(title)
                votes.append(helpful_votes)
            
            # Find and click the button to navigate to the next page
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, '#reviews-accordion > div.page-pagination.mb-300 > div > div:nth-child(2) > ul > li.page.next > a'))
            )
            next_button.click()
            time.sleep(3)

            all_star_rating.extend(star_rating)
            all_review.extend(review)
            all_name.extend(name)
            all_review_title.extend(review_title)
            all_votes.extend(votes)
        
        except NoSuchElementException:
            # If no next button is found, exit the loop
            break

    


Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[2]/main/div[1]/div/div/div[2]/div/div/div/div/div[1]/ul/li[1]/div/div[2]/div[5]/p"}
  (Session info: chrome=122.0.6261.69); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6F6C05E42+3538674]
	(No symbol) [0x00007FF6F6824C02]
	(No symbol) [0x00007FF6F66D5AEB]
	(No symbol) [0x00007FF6F671BF4E]
	(No symbol) [0x00007FF6F671C0CC]
	(No symbol) [0x00007FF6F675E477]
	(No symbol) [0x00007FF6F673F0EF]
	(No symbol) [0x00007FF6F675BDE3]
	(No symbol) [0x00007FF6F673EE53]
	(No symbol) [0x00007FF6F670F514]
	(No symbol) [0x00007FF6F6710631]
	GetHandleVerifier [0x00007FF6F6C36CAD+3738973]
	GetHandleVerifier [0x00007FF6F6C8C506+4089270]
	GetHandleVerifier [0x00007FF6F6C84823+4057299]
	GetHandleVerifier [0x00007FF6F6955C49+720121]
	(No symbol) [0x00007FF6F683126F]
	(No symb

ElementClickInterceptedException: Message: element click intercepted: Element <a aria-disabled="false" class="" data-track="Page next" href="/site/reviews/norton-360-deluxe-5-device-antivirus-internet-security-software-vpn-dark-web-monitoring-1-year-subscription-android-mac-os-windows-apple-ios-digital/6346690?variant=A&amp;page=16&amp;page=17&amp;pageSize=20&amp;sku=6346690&amp;sort=BEST_REVIEW&amp;variant=A" role="button" title="next Page">...</a> is not clickable at point (1435, 54). Other element would receive the click: <div class="cart-icon">...</div>
  (Session info: chrome=122.0.6261.69)
Stacktrace:
	GetHandleVerifier [0x00007FF6F6C05E42+3538674]
	(No symbol) [0x00007FF6F6824C02]
	(No symbol) [0x00007FF6F66D5AEB]
	(No symbol) [0x00007FF6F67233E4]
	(No symbol) [0x00007FF6F6721122]
	(No symbol) [0x00007FF6F671E884]
	(No symbol) [0x00007FF6F671D5A9]
	(No symbol) [0x00007FF6F6711950]
	(No symbol) [0x00007FF6F673F0AA]
	(No symbol) [0x00007FF6F671124A]
	(No symbol) [0x00007FF6F673F2C

In [16]:
match = re.search(r'\((\d{2,4})\sReviews\)', tot_reviews)
review_count = match.group(1)
count = int(review_count)
count
num_pages = round(count/20)
num_pages

12

In [ ]:
star = driver.find_element(By.XPATH, f'//*[@id="reviews-accordion"]/div[1]/ul/li[{i}]/div/div[2]/div[1]/div/div/p').text[6:7]
rev = driver.find_element(By.XPATH, f'/html/body/div[2]/main/div[1]/div/div/div[2]/div/div/div/div/div[1]/ul/li[{i}]/div/div[2]/div[5]/p').text
rev_name = driver.find_element(By.XPATH, f'//*[@id="reviews-accordion"]/div[1]/ul/li[{i}]/div/div[1]/div/span/span/button/div').text
title = driver.find_element(By.XPATH, f'/html/body/div[2]/main/div[1]/div/div/div[2]/div/div/div/div/div[1]/ul/li[{i}]/div/div[2]/div[1]/h4').text
helpful_votes = driver.find_element(By.XPATH, f'/html/body/div[2]/main/div[1]/div/div/div[2]/div/div/div/div/div[1]/ul/li[{i}]/div/div[2]/div[7]/div/div/div[1]/button[1]').text.split('(')[1].split(')')[0]



In [17]:
for i in range(1,num_pages):
    try:
        # Retrieve review elements
        review_elements = driver.find_elements(By.XPATH, '//*[@id="reviews-accordion"]/div[1]/ul/li')
        total_reviews = len(review_elements)

        # Loop through each review on the current page
        for i in range(1, total_reviews + 1):
            try:
                star = driver.find_element(By.XPATH, f'//*[@id="reviews-accordion"]/div[1]/ul/li[{i}]/div/div[2]/div[1]/div/div/p').text[6:7]
                rev = driver.find_element(By.XPATH, f'/html/body/div[2]/main/div[1]/div/div/div[2]/div/div/div/div/div[1]/ul/li[{i}]/div/div[2]/div[5]/p').text
                rev_name = driver.find_element(By.XPATH, f'//*[@id="reviews-accordion"]/div[1]/ul/li[{i}]/div/div[1]/div/span/span/button/div').text
                title = driver.find_element(By.XPATH, f'/html/body/div[2]/main/div[1]/div/div/div[2]/div/div/div/div/div[1]/ul/li[{i}]/div/div[2]/div[1]/h4').text
                helpful_votes = driver.find_element(By.XPATH, f'/html/body/div[2]/main/div[1]/div/div/div[2]/div/div/div/div/div[1]/ul/li[{i}]/div/div[2]/div[7]/div/div/div[1]/button[1]').text.split('(')[1].split(')')[0]
            except Exception as e:
                print(e)

            star_rating.append(star)
            review.append(rev)
            name.append(rev_name)
            review_title.append(title)
            votes.append(helpful_votes)
        
        # Find and click the button to navigate to the next page
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, '#reviews-accordion > div.page-pagination.mb-300 > div > div:nth-child(2) > ul > li.page.next > a'))
        )
        next_button.click()
        time.sleep(3)
    except NoSuchElementException:
        # If no next button is found, exit the loop
        break

Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[2]/main/div[1]/div/div/div[2]/div/div/div/div/div[1]/ul/li[1]/div/div[2]/div[5]/p"}
  (Session info: chrome=122.0.6261.69); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6F6C05E42+3538674]
	(No symbol) [0x00007FF6F6824C02]
	(No symbol) [0x00007FF6F66D5AEB]
	(No symbol) [0x00007FF6F671BF4E]
	(No symbol) [0x00007FF6F671C0CC]
	(No symbol) [0x00007FF6F675E477]
	(No symbol) [0x00007FF6F673F0EF]
	(No symbol) [0x00007FF6F675BDE3]
	(No symbol) [0x00007FF6F673EE53]
	(No symbol) [0x00007FF6F670F514]
	(No symbol) [0x00007FF6F6710631]
	GetHandleVerifier [0x00007FF6F6C36CAD+3738973]
	GetHandleVerifier [0x00007FF6F6C8C506+4089270]
	GetHandleVerifier [0x00007FF6F6C84823+4057299]
	GetHandleVerifier [0x00007FF6F6955C49+720121]
	(No symbol) [0x00007FF6F683126F]
	(No symb

In [22]:
reviews = pd.DataFrame({'Reviewer_Name':name,'Review_Title':review_title, 'Review_content':review ,'Star_Rating':star_rating})
reviews.head()

,Reviewer_Name,Review_Title,Review_content,Star_Rating
0,MadhankumarM,McAfee - Best in the market,"Worth , VPN is also very affective ...",5
1,ReginaldG,MC AFEE,It is great protection\nThe best i had by far\...,5
2,MasterChief,Great price great PC security and decent VPN,Good security for my phones and PC especially ...,4
3,BuyingDeals,Buy it every year,I have been purchasing McAfee Total Protection...,5
4,ShadeB,Great laptop/great service,I strolled in with bare minimum time left to m...,5


In [5]:
driver.quit()